In [1]:
using Pkg; Pkg.activate("C:\\Users\\KarlH\\Desktop\\ComputationalPhysicsMaster\\Code\\OrbitalNeuralMethods")
using Revise
using OrbitalNeuralMethods

  Activating project at `C:\Users\KarlH\Desktop\ComputationalPhysicsMaster\Code\OrbitalNeuralMethods`


In [2]:
import Random
rng = Random.MersenneTwister();

In [3]:
n = 6
ω = 0.25
ham = HOCoulomb(ω, shielding = 0.25);

l = 20
basis = SpinBasis(HOBasis(l, ω))
grid = [x for x in range(-10, stop = 10, length = 2001)]
system = System(n, basis, grid, ham);

mixer = Alpha(0.5)
rhf = RHF(system, mixer);
rhf = compute_ground_state!(rhf, max_iter = 1000, tol = 1e-10, verbose=0);
energy(rhf)

9.568072208063928

In [65]:
metro = Importance(equils = 1e4, samples = 1e5, step = 0.01);
#metro = Metropolis(equils = 1e4, samples = samples, step = 0.1);

In [53]:
nn = NeuralNetwork([Dense(60), Tanh, Dense(12), Tanh, Dense(1), Exp], n=n, rng = rng);
slater = Slater(rhf);
wf = SlaterNN(slater, nn);
optimizer = ADAM(wf, lr = 0.01, max_iter = 10, tol = 1e-6);

In [66]:
@time wf, grad_results = optimize(wf, ham, metro, optimizer, nthreads = 12, verbose = true);

E = 9.236 iter = 10/10                                      
No convergence reached, final norm of gradient was 0.0899059505915135
  2.851424 seconds (35.64 k allocations: 26.995 MiB)


In [67]:
samples_b = 2^24
metro_b = Importance(equils = 1e5, samples = samples_b, step = 0.01);
@time result = blocking(wf, ham, metro_b, nthreads = 12);
result.E, samples_b

 25.988489 seconds (9.12 k allocations: 1.377 GiB, 1.16% gc time, 0.04% compilation time)


(9.549128323447214, 16777216)